In [1]:
import pyscf
import pyscf.tools
import sys
sys.path.append('../../../OrbitalPartitioning/orbitalpartitioning')
import orbitalpartitioning
from orbitalpartitioning import *

In [2]:
 molecule = '''
 C       4.6336867757    -1.2242295397     0.0000000000
 C       4.4255614400     0.1700917199     0.0000000000
 C       5.9160648559    -1.7742824642     0.0000000000
 C       5.5765818320     0.9841959654     0.0000000000
 C       7.0417938838    -0.9473143671     0.0000000000
 C       6.8604981940     0.4376683883     0.0000000000
 C       0.5927492072     0.4581981628     0.0000000000
 C       0.4451033014     1.8584342989     0.0000000000
 C       1.9041039939    -0.0505338987     0.0000000000
 C       1.5640941394     2.6887327507     0.0000000000
 C       3.0514687259     0.7633813258     0.0000000000
 C       2.8520613255     2.1570405955     0.0000000000
 C      -6.8604981940    -0.4376683883     0.0000000000
 C      -7.0417938838     0.9473143671     0.0000000000
 C      -5.5765818320    -0.9841959654     0.0000000000
 C      -5.9160648559     1.7742824642     0.0000000000
 C      -4.4255614400    -0.1700917199     0.0000000000
 C      -4.6336867757     1.2242295397     0.0000000000
 C      -3.0514687259    -0.7633813258     0.0000000000
 C      -2.8520613255    -2.1570405955     0.0000000000
 C      -1.5640941394    -2.6887327507     0.0000000000
 C      -0.4451033014    -1.8584342989     0.0000000000
 C      -0.5927492072    -0.4581981628     0.0000000000
 C      -1.9041039939     0.0505338987     0.0000000000
 H       3.7849879120    -1.9076345469     0.0000000000
 H       6.0325833442    -2.8604965477     0.0000000000
 H       5.4821758879     2.0696091478     0.0000000000
 H       8.0463585128    -1.3756739224     0.0000000000
 H      -0.5426199813     2.3168156796     0.0000000000
 H       2.0368540411    -1.1280906363     0.0000000000
 H       1.4298106521     3.7726953568     0.0000000000
 H       3.6983932318     2.8425398416     0.0000000000
 H      -8.0463585128     1.3756739224     0.0000000000
 H      -5.4821758879    -2.0696091478     0.0000000000
 H      -6.0325833442     2.8604965477     0.0000000000
 H      -3.7849879120     1.9076345469     0.0000000000
 H      -3.6983932318    -2.8425398416     0.0000000000
 H      -1.4298106521    -3.7726953568     0.0000000000
 H       0.5426199813    -2.3168156796     0.0000000000
 H      -2.0368540411     1.1280906363     0.0000000000
 H      -7.7260669052    -1.1042415323     0.0000000000
 H       7.7260669052     1.1042415323     0.0000000000
 '''
basis = "cc-pvdz"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        #symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
#print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.RHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
#mf.conv_tol_grad = 1e-5
#mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
print("size of MO coeff", C.shape)
ndocc = pymol.nelec[1]
nsing = pymol.nelec[0] - ndocc
nvirt = pymol.mol.nao - ndocc - nsing
print(ndocc)
print(nsing)
print(nvirt)
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/td/qpnnxwv93pq0t7bbdkh5rvzr0000gn/T/tmpy_is74az
max_memory 4000 MB (current use 0 MB)
Set gradient conv threshold to 0.0001
init E= -927.106444075062
  HOMO = -0.193491983172024  LUMO = -0.00869116094500147
cycle= 1 E= -919.136142169225  delta_E= 7.97  |g|= 0.802  |ddm|= 7.16
  HOMO = -0.307049220455122  LUMO = 0.0471331049734749
cycle= 2 E= -919.382826658245  delta_E= -0.247  |g|= 0.227  |ddm|=  1.5
  HOMO = -0.271000955386479  LUMO = 0.0878913637119213
cycle= 3 E= -919.400784324615  delta_E= -0.018  |g|= 0.102  |ddm|= 0.647
  HOMO = -0.283241721033322  LUMO = 0.0771079206307622
cycle= 4 E= -919.403806996733  delta_E= -0.00302  |g|= 0.0199  |d

In [7]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3,4,5):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                print(ao)
                frag1.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (6,7,8,9,10,11):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                frag2.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (12,13,14,15,16,17):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                frag3.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (18,19,20,21,22,23):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                frag4.append(ao_idx)
                full.append(ao_idx)            


frags = [frag1, frag2, frag3, frag4]
print(frags)
print(len(frag1))
print(len(frag2))
print(len(frag3))
print(len(frag4))

(0, 'C', '2p', 'z')
(1, 'C', '2p', 'z')
(2, 'C', '2p', 'z')
(3, 'C', '2p', 'z')
(4, 'C', '2p', 'z')
(5, 'C', '2p', 'z')
[[5, 19, 33, 47, 61, 75], [89, 103, 117, 131, 145, 159], [173, 187, 201, 215, 229, 243], [257, 271, 285, 299, 313, 327]]
6
6
6
6


In [8]:
# Define projectors
nbas = Cdocc.shape[0]
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])

### Project MOs onto all Fragments

In [9]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

 Partition  426 orbitals into a total of   24 orbitals
            Index   Sing. Val. Space       
                0   1.13290899            2*
                1   1.12584583            2*
                2   1.11594105            2*
                3   1.10688827            2*
                4   1.07548368            2*
                5   1.06809999            2*
                6   1.06387181            2*
                7   1.05627062            2*
                8   1.05626129            2*
                9   1.04602876            2*
               10   1.04179759            2*
               11   1.03833321            2*
               12   0.88664875            0*
               13   0.88506595            0*
               14   0.87716172            0*
               15   0.86634779            0*
               16   0.86437833            0*
               17   0.86021462            0*
               18   0.85570422            0*
               19   0.85241699            0*
 

In [60]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    # (Of, Sf, Vf), (Oact, Sact, Vact) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)

#np.save("Cfrags", Cfrags)

Cfrags = np.load("C_nat_orbs_sym_pc.npy")
#print("natrual orbs!")
Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
#pyscf.tools.molden.from_mo(mf.mol, "Pfull_no.molden", Pfull)
#pyscf.tools.molden.from_mo(mf.mol, "Cact_no.molden", Cact)
#pyscf.tools.molden.from_mo(mf.mol, "Cenv_no.molden", Cenv)
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag_sym_pc%i.molden"%i, Cfrags[i])
    #pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])


print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Fragment:  [5, 19, 33, 47, 61, 75]
 Partition   24 orbitals into a total of    6 orbitals
            Index   Sing. Val. Space       
                0   1.11389298            2*
                1   1.05620901            2*
                2   1.04269091            2*
                3   0.86506649            0*
                4   0.86371801            0*
                5   0.82528113            0*
                6   0.15486751            0
                7   0.11841518            2
                8   0.01134406            0
                9   0.01034068            2
               10   0.00268253            0
               11   0.00153611            2

 Fragment:  [89, 103, 117, 131, 145, 159]
 Partition   24 orbitals into a total of    6 orbitals
            Index   Sing. Val. Space       
                0   1.11029559            2*
                1   1.04933411            2*
                2   1.03773033            2*
                3   0.86433506            0*
        

## Make Integrals

In [61]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0_ras_no_sympc", h0)
np.save("ints_h1_ras_no_sympc", h1)
np.save("ints_h2_ras_no_sympc", h2)
np.save("mo_coeffs_ras_no_sympc", Cact)
np.save("overlap_mat_ras_no_sympc", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
#np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
#np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(426, 69)
(426, 24)
(426, 426)


### Try to make Natural Orbs

In [51]:
P = mf.make_rdm1()
P_act = Cact.T @ S @ P @ S @ Cact
e, v = np.linalg.eigh(P_act)
print(e)
Cact_no = Cact @ v
pyscf.tools.molden.from_mo(mf.mol, "Cact_no.molden", Cact_no)

[-1.38033821e-15 -1.02654039e-15 -7.01132421e-16 -2.34862950e-16
 -8.32139141e-17  1.33787441e-16  2.35892843e-16  4.49656052e-16
  4.80842634e-16  5.16255709e-16  6.52326463e-16  1.02833050e-15
  2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00
  2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00
  2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00]


In [49]:
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact_no.T @ h @ Cact_no;
j = Cact_no.T @ j @ Cact_no;
k = Cact_no.T @ k @ Cact_no;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact_no, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact_no @ Cact_no.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0_no", h0)
np.save("ints_h1_no", h1)
np.save("ints_h2_no", h2)
np.save("mo_coeffs_no", Cact_no)
np.save("overlap_mat_no", S)


(426, 426)
